<a href="https://colab.research.google.com/github/2320030030/Aiml---Project/blob/main/Updated%20Sample%20project%20Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import cv2
import os
import numpy as np
import datetime

# Load the pre-trained Haar Cascade model for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Example dataset: Pre-cropped grayscale face images stored in "dataset" folder
# The names are linked to file paths
registered_faces = {
    "Indira Priyadarshini": "dataset/indira.jpg",
    "Venkat Reddy": "dataset/venkat.jpg",
    "Shanmukha Devi": "dataset/shanmukha.jpg",
    "Keerthi Dhara Reddy": "dataset/keerthi.jpg"
}

# Load dataset into memory (converted to grayscale for matching)
def load_dataset():
    dataset = {}
    for name, filepath in registered_faces.items():
        img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)  # Load as grayscale
        if img is not None:
            dataset[name] = img
    return dataset

# Function to mark attendance
def mark_attendance(name, attendance):
    current_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    if name not in attendance:
        attendance[name] = current_time
        print(f"{name} marked present at {current_time}")

# Function to compare detected face with dataset images (basic image matching)
def recognize_face(face, dataset, threshold=5000):
    face = cv2.resize(face, (160, 160))  # Resize detected face for comparison
    min_dist = float('inf')
    identity = None
    for name, registered_face in dataset.items():
        registered_face = cv2.resize(registered_face, (160, 160))  # Ensure same size
        diff = cv2.absdiff(face, registered_face)  # Compute absolute difference
        dist = np.sum(diff)  # Sum of differences as a basic distance metric
        if dist < min_dist and dist < threshold:
            min_dist = dist
            identity = name
    return identity

# Real-time attendance system
def run_attendance_system():
    attendance = {}
    dataset = load_dataset()  # Load the example dataset
    cap = cv2.VideoCapture(0)  # Access webcam

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Convert frame to grayscale (for Haar Cascade detection)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces in the frame
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(100, 100))

        for (x, y, w, h) in faces:
            face = gray[y:y+h, x:x+w]  # Extract the detected face

            # Try to recognize the face by matching it with the dataset
            name = recognize_face(face, dataset)

            # Mark attendance if face is recognized
            if name:
                mark_attendance(name, attendance)
                color = (0, 255, 0)  # Green for recognized faces
            else:
                name = "Unknown"
                color = (0, 0, 255)  # Red for unknown faces

            # Draw rectangle and label around detected face
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            cv2.putText(frame, name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

        # Show video feed with detected and labeled faces
        cv2.imshow('Attendance System', frame)

        # Exit loop on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Main execution
if __name__ == '__main__':
    run_attendance_system()
